In [8]:
import numpy as np
import pandas as pd
import os, re, operator
from pprint import pprint

from src.utils import *
from src.k_nearest_neighbors import *
from src.logistic_regression import *
from src.naive_bayes import *
from src.neutral_network import *
#from src.support_vector_machines import *
from src.validation import *

PATH_POSITIVE_TRUTHFUL  = 'op_spam_v1.4/positive/truthful/'
PATH_POSITIVE_DECEPTIVE = 'op_spam_v1.4/positive/deceptive/'
PATH_NEGATIVE_TRUTHFUL  = 'op_spam_v1.4/negative/truthful/'
PATH_NEGATIVE_DECEPTIVE = 'op_spam_v1.4/negative/deceptive/'
ALL_PATH = [PATH_POSITIVE_TRUTHFUL, PATH_POSITIVE_DECEPTIVE,
           PATH_NEGATIVE_TRUTHFUL, PATH_NEGATIVE_DECEPTIVE]

In [9]:
# Remoção de Stop Words
import nltk
from nltk.corpus import stopwords

# Download do dicionário de stop wordsgreat
nltk.download('stopwords')

def stopwords_removal(tokens_list):
    new_tokens_list = []
    
    stop_words = stopwords.words('english')
    new_tokens_list = [token for token in tokens_list if token not in stop_words]
    
    return new_tokens_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# N-gram dos tokens
def generate_ngram(tokens_list, gram1, gram2):
    # N-grams list
    allNGrams = []
    
    #N-Gram
    for idx in range(gram1, gram2):
        ngrams = zip(*[tokens_list[i:] for i in range(idx)])
        allNGrams += ([" ".join(ngram) for ngram in ngrams])
    
#     print(allNGrams)
    return allNGrams

In [11]:
# Remoção de features irrelevantes por threshold
def features_removal(features_dic, threshold):
    new_features_dic = {}

    #Remove values < threshold
    for key, value in features_dic.items():
        if value > threshold:
            new_features_dic[key] = value

    new_features_list = [k for k, v in new_features_dic.items()]
    new_features_dic = sorted(new_features_dic.items(), key=operator.itemgetter(1))
    
    return new_features_list, new_features_dic

In [28]:
matrix = []
Y = []

def generateFeatures(token_list, token_per_example, example_class):    
    for tokens in token_per_example:
        row = np.zeros(len(token_list))
        for column, token in enumerate(token_list):
            if token in tokens:
                row[column] += 1

    Y.append(example_class)
    matrix.append(row)
            
generateFeatures(PATH_NEGATIVE_DECEPTIVE, 1)
generateFeatures(PATH_NEGATIVE_TRUTHFUL, 0)
# generateFeatures(PATH_POSITIVE_DECEPTIVE, 1)
# generateFeatures(PATH_POSITIVE_TRUTHFUL, 0)

matrix = np.array(matrix)
Y = np.array(Y)
# matrix_norm, mu, sigma = normalize(matrix)

#save(np.column_stack((matrix_norm, matrix[:, -1])), listWords)
print('ACABOOOOOOO')

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [27]:
# Tokenização dos textos
def pre_processing(paths, stopwords=False, ngram=False, threshold=0):
    token_per_example = []
    token_dic = {}
    dirs =[]
    
    for p in paths:
        text_path = [p + t for t in os.listdir(p)]
        dirs += (text_path)
    
    for texts in dirs:
        with open(texts, 'r', encoding='utf-8') as stream:
            tokens = []
            text = stream.read()

            #Pre processing
            tokens = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']
            
            if stopwords:
                tokens = stopwords_removal(tokens)
            
            if ngram:
                tokens = generate_ngram(tokens, 1, 3)

            token_per_example.append(tokens)
            
            #Creating dictonary for counting words
            for token in tokens:
                if token in token_dic:
                    token_dic[token] += 1
                else:
                    token_dic[token] = 1
    
    token_list, token_dic = features_removal(token_dic, threshold)

    generate_features(token_list, token_per_example)
    
    return token_list, token_dic

# posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE, PATH_NEGATIVE_DECEPTIVE], 7)
# posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE], 3)
posTruthDic, listWords = pre_processing([PATH_NEGATIVE_DECEPTIVE], stopwords=True, ngram=True, threshold=3)

#pprint(posTruthDic)
print(len(posTruthDic))

2069


In [12]:
# # Pre processamento
# def pre_processing(paths, stopwords=False, ngram=False, threshold=0):
#     tokens, tokens_dic = tokenize(paths)
    
#     if (stopwords):
#         tokens, tokens_dic = stopwords_removal(tokens)
    
#     if (ngram):
#         tokens, tokens_dic = generate_ngram(tokens, 1, 3)
    
#     tokens, tokens_dic = features_removal(tokens_dic, threshold)
    
#     return tokens, tokens_dic

# # posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE, PATH_NEGATIVE_DECEPTIVE], 7)
# # posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE], 3)
# posTruthDic, listWords = pre_processing([PATH_NEGATIVE_DECEPTIVE])

# #pprint(posTruthDic)
# print(len(posTruthDic))

In [20]:
display(matrix_norm)
if matrix_norm.shape[1] != len(posTruthDic):
    raise Exception('Tamanhos diferentes!')

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [ ]:
# a = matrix_norm[400:720]
# b = matrix_norm[0:320]
# X2 = np.append(a,b, axis=0)

# a = Y[400:720]
# b = Y[0:320]
# Y2 = np.append(a,b, axis=0)

# a = matrix_norm[720:800]
# b = matrix_norm[320:400]
# X_val = np.append(a,b, axis=0)

# a = Y[720:800]
# b = Y[320:400]
# Y_val = np.append(a,b, axis=0)

#custo, gamma = svm(X2, Y2, X_val, Y_val)
#print(custo, '\t', gamma)

In [ ]:
# y = knn(X_val[81], X2, Y2, 5)
# #print(Y_val[81], ' ',y,' ', ind_viz)
# print(y)

In [ ]:
folds = stratified_kfolds(Y, 5, np.unique(Y))

for train_index, test_index in folds:
    print('Train: ',len(train_index))
    print('Test: ',len(test_index))

In [ ]:
# import random

# resultados = []
# for x in range(0, 5):
#     Y_resultado = np.array([random.randint(0, 1) for i in range(0, 160)])
#     Y_classes = np.array([random.randint(0, 1) for i in range(0, 160)])
#     classes = [0, 1]

#     cm = get_confusionMatrix(Y_classes, Y_resultado, classes)
    
    
#     resultado = relatorioDesempenho(cm, classes, False)
#     resultados.append(resultado)

# mediaFolds(resultados, classes)

In [ ]:
# NAIVE BAYES

resultados = []
classes = np.unique(Y)

for train_index, test_index in folds:
    # Treinamento
    probsPos, probsNeg = calcularProbabilidades(matrix[train_index], Y[train_index])
    
    # Classificação
    pred = []
    for x in matrix[test_index]:
        probPos, probNeg = classificacao(x, probsPos, probsNeg, sum(Y == 1)/len(Y), sum(Y == 0)/len(Y))
        if (probPos >= probNeg):
            pred.append(1)
        else:
            pred.append(0)

    cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
    resultado = relatorioDesempenho(cm, classes, False)
    
    resultados.append(resultado)

mediaFolds(resultados, classes)

In [36]:
#print('s' in ['a','b','c'])
token_per_example = [['e','b','c'], ['a','b','c']]

#tokens = ['a','b','c']
token_list = ['s', 'c', 'e']

for tokens in token_per_example:
    for idx, token in enumerate(token_list):
        if token in tokens:
            print('yep', idx)
        else:
            print('nop', idx)
    print('\n')

nop 0
yep 1
yep 2


nop 0
yep 1
nop 2


